## Webscraping Bonus

### 1. Navigate to https://www.billboard.com/charts/hot-100/. Using BeautifulSoup, extract out the This Week, artist, song, Last Week, Peak Position, and Weeks on Chart values into a pandas DataFrame. Hint: The HTML for the number one ranked song is slightly different from that of the rest of the songs.

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import datetime
from datetime import timedelta


In [3]:
endpoint = 'https://www.billboard.com/charts/hot-100/'
response = requests.get(endpoint)
soup = BeautifulSoup(response.text, features='html.parser')

**The below commented out code was my first attempt at pulling each requested item by row, but I ran into indexing issues. I then backed out and realized i could do a larger for loop that would give my all of the information i was looking for except the 'This Week' column, because that was in a separate div and fell outside the scope of the for loop. However, the this_week list comprehension worked great, but it's also in index order more or less, so I could use the index as the 'This Week' column if I set the first value to 1 instead of 0.**

In [4]:
# titles = [row.h3.text.strip() for row in soup.select('.o-chart-results-list-row')]
# this_week = [row.span.text.strip() for row in soup.select('.o-chart-results-list-row')]
# artists = [row.ul.li.span.text.strip() for row in soup.select('.o-chart-results-list-row')]
# last_week = [row.ul.ul.span.text.strip() for row in soup.select('.o-chart-results-list-row')]

**This is my final solution**

In [5]:
# list comprehension to get 'This Week' values
this_week = [row.span.text.strip() for row in soup.select('.o-chart-results-list-row')]
# initialize list for use inside loop
here = []
for row in soup.select('.o-chart-results-list-row'):
    item = row.ul.text.strip() # gets the text info inside the unordered list
    item = re.sub(r'[\n\t]+', ',', item) # removes many \n and \t and replaces with commas
    here.append(item) #add each item to the list
# split the list into elements separated by commas rather than a list of 100 elements
data = [row.split(',') for row in here]
# create dataframe, clean, and reshape
info_df = (
    pd.DataFrame(data,columns = ['Song Title', 'Artist', 'Last Week A', 'Peak A', 'Wks on Chart A','Last Week B', 'Peak B', 'Wks on Chart B', 'None' ])
    .drop(columns=['Last Week B', 'Peak B', 'Wks on Chart B', 'None'])
    .rename(columns={'Last Week A':'Last Week', 'Peak A':'Peak', 'Wks on Chart A':'Weeks on Chart'})
)
info_df['This Week'] = this_week
info_df['Last Week'] = [re.sub(r'-+', 'New to the list!', row) for row in info_df['Last Week']]
info_df = info_df.loc[:,['Song Title', 'Artist','This Week', 'Last Week', 'Peak', 'Weeks on Chart']]
info_df

,Song Title,Artist,This Week,Last Week,Peak,Weeks on Chart
0,Love Somebody,Morgan Wallen,1,New to the list!,1,1
1,A Bar Song (Tipsy),Shaboozey,2,1,1,28
2,Birds Of A Feather,Billie Eilish,3,2,2,23
3,Die With A Smile,Lady Gaga & Bruno Mars,4,4,3,10
4,Espresso,Sabrina Carpenter,5,3,3,28
...,...,...,...,...,...,...
95,Leave Me Alone,BigXthaPlug,96,99,96,2
96,Belong Together,Mark Ambor,97,New to the list!,74,24
97,The Emptiness Machine,Linkin Park,98,New to the list!,21,6
98,Mantra,Jennie,99,98,98,2


### 2. After getting the code working for the current chart, navigate to last week's chart. Notice how the url for the page changes. Write a function which will, given a date, return a pandas DataFrame containing the Billboard chart data for that date.

**Link format**
https://www.billboard.com/charts/hot-100/2024-10-26/

In [6]:
def get_billboard_chart(date=None):
    data=[] # initialize to reset each time function is run
    # default to current week if no date is given
    if date == None:
        date = datetime.date.today().strftime('%Y-%m-%d')
    endpoint = 'https://www.billboard.com/charts/hot-100/'+date+'/'
    response = requests.get(endpoint)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, features='html.parser')
        # Loop through each row of the chart and extract relevant text data
        for row in soup.select('.o-chart-results-list-row'):
            item = row.ul.text.strip()  # Extract text inside the unordered list
            item = re.sub(r'[\n\t]+', ',', item)  # Replace newline and tab characters with commas
            row_data = item.split(',')

            # Adjust row length to a specific number of columns (e.g., 7)
            if len(row_data) > 7:
                row_data = row_data[:7]  # Truncate to 7 items if longer
            elif len(row_data) < 7:
                row_data.extend([None] * (7 - len(row_data)))  # Pad with None if shorter

            data.append(row_data)  # Add processed row data to the list

        info_df = (
            pd.DataFrame(data,columns = ['Song Title', 'Artist', 'Last Week A', 'Peak A', 'Wks on Chart A','Last Week B', 'Peak B'])
            .drop(columns=['Last Week B', 'Peak B'])
            .rename(columns={'Last Week A':'Last Week', 'Peak A':'Peak', 'Wks on Chart A':'Weeks on Chart'})
        )
        info_df = info_df.rename_axis(index='This Week')
        info_df.index += 1
        info_df['Last Week'] = [re.sub(r'-+', 'New to the list!', row) for row in info_df['Last Week']]
        info_df = info_df.loc[:,['Song Title', 'Artist', 'Last Week', 'Peak', 'Weeks on Chart']]
    else:
        print(f"Request failed with status code: {response.status_code}")
    return info_df

In [7]:
# date = '2024-02-10'
get_billboard_chart()

,Song Title,Artist,Last Week,Peak,Weeks on Chart
This Week,,,,,
1,Love Somebody,Morgan Wallen,New to the list!,1,1
2,A Bar Song (Tipsy),Shaboozey,1,1,28
3,Birds Of A Feather,Billie Eilish,2,2,23
4,Die With A Smile,Lady Gaga & Bruno Mars,4,3,10
5,Espresso,Sabrina Carpenter,3,3,28
...,...,...,...,...,...
96,Leave Me Alone,BigXthaPlug,99,96,2
97,Belong Together,Mark Ambor,New to the list!,74,24
98,The Emptiness Machine,Linkin Park,New to the list!,21,6


### 3. Write a loop to retrieve the Billboard chart data for the last 10 weeks.

In [8]:
# Get today's date
today = datetime.date.today()
dfs = []
# Iterate 10 weeks back
for i in range(10):
    # Calculate the date for the week
    week_date = today - timedelta(weeks=i)
    # Format the date as a string (e.g., YYYY-MM-DD)
    date_string = week_date.strftime("%Y-%m-%d")
    dfs.append(get_billboard_chart(date_string))
print(dfs)
display(dfs[3])
display(dfs[8])

[                       Song Title                  Artist         Last Week  \
This Week                                                                     
1                   Love Somebody           Morgan Wallen  New to the list!   
2              A Bar Song (Tipsy)               Shaboozey                 1   
3              Birds Of A Feather           Billie Eilish                 2   
4                Die With A Smile  Lady Gaga & Bruno Mars                 4   
5                        Espresso       Sabrina Carpenter                 3   
...                           ...                     ...               ...   
96                 Leave Me Alone             BigXthaPlug                99   
97                Belong Together              Mark Ambor  New to the list!   
98          The Emptiness Machine             Linkin Park  New to the list!   
99                         Mantra                  Jennie                98   
100        Angel With An Attitude                Ro

,Song Title,Artist,Last Week,Peak,Weeks on Chart
This Week,,,,,
1,A Bar Song (Tipsy),Shaboozey,1,1,25
2,Birds Of A Feather,Billie Eilish,6,2,20
3,Timeless,The Weeknd & Playboi Carti,New to the list!,3,1
4,I Had Some Help,Post Malone Featuring Morgan Wallen,2,1,21
5,Espresso,Sabrina Carpenter,3,3,25
...,...,...,...,...,...
96,Keep Up,Odetari,New to the list!,96,1
97,Passport Junkie,Rod Wave,98,61,3
98,This Is My Dirt,Justin Moore,New to the list!,98,1


,Song Title,Artist,Last Week,Peak,Weeks on Chart
This Week,,,,,
1,A Bar Song (Tipsy),Shaboozey,1,1,20
2,Taste,Sabrina Carpenter,New to the list!,2,1
3,Please Please Please,Sabrina Carpenter,9,1,12
4,Espresso,Sabrina Carpenter,7,3,20
5,I Had Some Help,Post Malone Featuring Morgan Wallen,2,1,16
...,...,...,...,...,...
96,Prove It,21 Savage & Summer Walker,New to the list!,43,8
97,American Nights,Zach Bryan,100,21,8
98,She's Somebody's Daughter (Reimagined),Drew Baldridge,96,93,3
